<a href="https://colab.research.google.com/github/rasecfaria/FinalProject/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `Sistema de Recomendação de Filmes` 🎬

Este notebook implementa um sistema de recomendação de filmes utilizando dois métodos diferentes:
1. **Filtragem colaborativa** - baseada nas avaliações dos usuários
2. **Recomendação baseada em conteúdo** - utilizando gêneros e tags dos filmes

O dataset utilizado é o [MovieLens](https://grouplens.org/datasets/movielens/) que contém avaliações de filmes feitas por usuários. Este notebook demonstra conceitos importantes de sistemas de recomendação, que podem ser aplicados em diferentes contextos.

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import os

## 1. Carregamento e Análise dos Dados

Primeiro, vamos carregar os datasets e explorar sua estrutura.

In [ ]:
def carregar_dados(diretorio_dados='ml-latest-small'):
    """Carrega os datasets necessários para o sistema de recomendação"""
    print("Carregando dados...")
    
    # Caminhos para os arquivos
    path_movies = os.path.join(diretorio_dados, 'movies.csv')
    path_ratings = os.path.join(diretorio_dados, 'ratings.csv')
    path_tags = os.path.join(diretorio_dados, 'tags.csv')
    path_dados = os.path.join(diretorio_dados, 'dados.csv')
    
    # Carregar datasets principais
    movies = pd.read_csv(path_movies)
    ratings = pd.read_csv(path_ratings)
    
    # Remover timestamp dos ratings pois não será utilizado
    ratings = ratings[['userId', 'movieId', 'rating']]
    
    # Informações sobre os datasets carregados
    print(f"Dimensões do dataset de filmes: {movies.shape}")
    print(f"Dimensões do dataset de avaliações: {ratings.shape}")
    
    # Carregar datasets adicionais para recomendação baseada em conteúdo
    try:
        tags = pd.read_csv(path_tags)
        dados = pd.read_csv(path_dados)
        print(f"Dimensões do dataset de tags: {tags.shape}")
        return movies, ratings, tags, dados
    except Exception as e:
        print(f"Aviso: Não foi possível carregar dados adicionais: {e}")
        return movies, ratings, None, None

# Carregar os datasets
movies, ratings, tags, dados = carregar_dados()

# Exibir exemplos dos datasets
print("\nPrimeiras linhas do dataset de filmes:")
movies.head()

In [ ]:
# Exibir as primeiras linhas do dataset de avaliações
print("Dataset de avaliações:")
ratings.head()

In [ ]:
# Análise do número de avaliações por usuário
usuario_ratings = ratings['userId'].value_counts()

print(f"Número médio de avaliações por usuário: {usuario_ratings.mean():.2f}")
print(f"Número máximo de avaliações feitas por um usuário: {usuario_ratings.max()}")
print(f"Número mínimo de avaliações feitas por um usuário: {usuario_ratings.min()}")

# Visualizar a distribuição de avaliações pelos top 10 usuários mais ativos
usuario_ratings.head(10)

(100836, 3)

In [ ]:
# Juntar os datasets de filmes e avaliações para análise
df = movies.merge(ratings, on='movieId')
print(f"Dimensões do dataframe mesclado: {df.shape}")
df.head()

## 2. Método de Filtragem Colaborativa

A filtragem colaborativa baseia-se na premissa de que usuários que concordaram no passado também concordarão no futuro. Neste caso, vamos usar a técnica de item-item collaborative filtering, onde comparamos a similaridade entre filmes baseado nas avaliações dadas pelos usuários.

In [ ]:
def preparar_dados_colaborativos(movies, ratings):
    """Prepara os dados para filtragem colaborativa"""
    print("Preparando dados para filtragem colaborativa...")
    
    # Mesclar datasets de filmes e avaliações
    df = movies.merge(ratings, on='movieId')
    
    # Criar tabela pivot: filmes x usuários com valores de avaliações
    movies_table = df.pivot_table(index='title', columns='userId', values='rating').fillna(0)
    
    print(f"Tabela pivot criada com dimensões: {movies_table.shape}")
    return movies_table

# Criar tabela pivot para filtragem colaborativa
movies_table = preparar_dados_colaborativos(movies, ratings)
movies_table.head(5)

In [ ]:
# Importar função para calcular similaridade
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def filtragem_colaborativa(movies_table, filme_referencia='Forrest Gump'):
    """Implementa o método de filtragem colaborativa usando similaridade de cosseno"""
    print(f"Calculando filmes similares a '{filme_referencia}' usando filtragem colaborativa...")
    
    # Calcular similaridade de cosseno entre todos os filmes
    rec = cosine_similarity(movies_table)
    rec_df = pd.DataFrame(rec, columns=movies_table.index, index=movies_table.index)
    
    # Filtrar e ordenar filmes similares ao filme de referência
    if filme_referencia in rec_df.columns:
        cossine_df = pd.DataFrame(rec_df[filme_referencia].sort_values(ascending=False))
        cossine_df.columns = ['recomendações']
        return cossine_df
    else:
        print(f"Erro: Filme '{filme_referencia}' não encontrado no dataset.")
        return None

# Calcular similaridade de cosseno entre todos os filmes
# Isso indica a similaridade entre cada filme, onde valores próximos de 1 indicam alta similaridade
rec = cosine_similarity(movies_table)
rec_df = pd.DataFrame(rec, columns=movies_table.index, index=movies_table.index)
print("Amostra da matriz de similaridade:")
rec_df.iloc[:5, :5]

In [ ]:
# Obter recomendações para um filme específico usando filtragem colaborativa
filme_referencia = 'Forrest Gump'
recomendacoes_colaborativas = filtragem_colaborativa(movies_table, filme_referencia)

print(f"\nTop 10 recomendações para '{filme_referencia}' usando filtragem colaborativa:")
recomendacoes_colaborativas.head(10)

## 3. Método Baseado em Conteúdo

O método de recomendação baseado em conteúdo utiliza características dos filmes para encontrar similaridades. Neste caso, usaremos principalmente os gêneros e, quando disponíveis, as tags dos filmes.

In [ ]:
# Importar bibliotecas para processamento de texto e similaridade
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
if tags is not None:
    # Visualizar as primeiras linhas do dataset de tags
    print("Dataset de tags:")
    tags.head()
else:
    print("Dataset de tags não disponível")

In [ ]:
def preparar_dados_conteudo(movies, tags, dados):
    """Prepara os dados para recomendação baseada em conteúdo"""
    print("Preparando dados para recomendação baseada em conteúdo...")
    
    # Converter movieId para string para garantir o merge correto
    movies['movieId'] = movies['movieId'].apply(lambda x: str(x))
    
    # Converter movieId em tags também para evitar erros de tipo
    tags_copy = tags.copy()
    tags_copy['movieId'] = tags_copy['movieId'].apply(lambda x: str(x))
    
    # Mesclar datasets para criar um único DataFrame enriquecido
    df2 = movies.merge(dados, left_on='title', right_on='Name', how='left')
    df2 = df2.merge(tags_copy, left_on='movieId', right_on='movieId', how='left')
    
    # Usar gêneros como base para informações de conteúdo
    # Poderia ser expandido para incluir descrições e tags
    df2['Infos'] = df2['genres']  # + str(df2['Discription']) + df2['tag']
    
    print(f"Dataset baseado em conteúdo preparado com dimensões: {df2.shape}")
    return df2

# Converter movieId para string para garantir o merge correto
movies['movieId'] = movies['movieId'].apply(lambda x: str(x))

(45256, 2)

In [ ]:
if tags is not None and dados is not None:
    # Preparar dataset para recomendação baseada em conteúdo
    df2 = preparar_dados_conteudo(movies, tags, dados)
    df2.head()
else:
    print("Dados insuficientes para recomendação baseada em conteúdo")

In [ ]:
def recomendacao_baseada_em_conteudo(df2, filme_referencia='Forrest Gump'):
    """Implementa o método de recomendação baseada em conteúdo usando TF-IDF e similaridade de cosseno"""
    print(f"Calculando filmes similares a '{filme_referencia}' baseado em conteúdo...")
    
    # Criar matriz TF-IDF
    vec = TfidfVectorizer()
    tfidf = vec.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))
    print(f"Matriz TF-IDF criada com dimensões: {tfidf.shape}")
    
    # Calcular similaridade de cosseno
    sim = cosine_similarity(tfidf)
    sim_df2 = pd.DataFrame(sim, columns=df2['title'], index=df2['title'])
    
    # Filtrar e ordenar filmes similares ao filme de referência
    if filme_referencia in sim_df2.columns:
        final_df = pd.DataFrame(sim_df2[filme_referencia].sort_values(ascending=False))
        final_df.columns = ['recomendações']
        return final_df
    else:
        print(f"Erro: Filme '{filme_referencia}' não encontrado no dataset.")
        return None

if tags is not None and dados is not None:
    # Criar matriz TF-IDF para análise de similaridade de texto
    vec = TfidfVectorizer()
    tfidf = vec.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))
    print(f"Matriz TF-IDF criada com dimensões: {tfidf.shape}")

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",Comedy


In [ ]:
if tags is not None and dados is not None:
    # Calcular similaridade de cosseno entre conteúdos dos filmes
    sim = cosine_similarity(tfidf)
    print(f"Matriz de similaridade baseada em conteúdo criada com dimensões: {sim.shape}")

In [ ]:
if tags is not None and dados is not None:
    # Criar DataFrame com matriz de similaridade
    sim_df2 = pd.DataFrame(sim, columns=df2['title'], index=df2['title'])
    print("Amostra da matriz de similaridade baseada em conteúdo:")
    sim_df2.iloc[:5, :5]

In [ ]:
def mostrar_recomendacoes(recomendacoes, n=10, filme_referencia=None):
    """Exibe as recomendações de filmes"""
    if recomendacoes is not None and not recomendacoes.empty:
        print(f"\nTop {n} recomendações para '{filme_referencia}':")
        return recomendacoes.head(n)
    else:
        print("Não foi possível gerar recomendações.")
        return None

if tags is not None and dados is not None:
    # Obter recomendações para um filme específico usando conteúdo
    filme_referencia = 'Forrest Gump'
    recomendacoes_conteudo = recomendacao_baseada_em_conteudo(df2, filme_referencia)
    
    print(f"\nTop 10 recomendações para '{filme_referencia}' baseadas em conteúdo:")
    mostrar_recomendacoes(recomendacoes_conteudo, 10, filme_referencia)

## 4. Comparação dos Métodos

Vamos comparar os resultados dos dois métodos para ver como diferem nas recomendações para o mesmo filme.

In [ ]:
# Função para comparar os resultados dos dois métodos
def comparar_recomendacoes(filme_referencia='Forrest Gump', n=10):
    """Compara as recomendações dos dois métodos para um mesmo filme"""
    print(f"\nComparando recomendações para '{filme_referencia}':")
    print("-" * 50)
    
    # Obter recomendações usando filtragem colaborativa
    recomendacoes_colaborativas = filtragem_colaborativa(movies_table, filme_referencia)
    
    if tags is not None and dados is not None:
        # Obter recomendações usando método baseado em conteúdo
        recomendacoes_conteudo = recomendacao_baseada_em_conteudo(df2, filme_referencia)
        
        # Criar um DataFrame para comparação
        comparacao = pd.DataFrame({
            'Filtragem Colaborativa': recomendacoes_colaborativas.head(n).index,
            'Baseada em Conteúdo': recomendacoes_conteudo.head(n).index
        })
        
        return comparacao
    else:
        print("Não foi possível comparar os métodos. Método baseado em conteúdo não disponível.")
        return None

# Comparar recomendações para um filme específico
if 'df2' in locals():
    comparacao = comparar_recomendacoes('Toy Story')
    if comparacao is not None:
        comparacao

## Conclusão

Neste notebook, implementamos dois métodos diferentes de recomendação de filmes:

1. **Filtragem Colaborativa**: Baseada nas avaliações dos usuários, usando similaridade de cosseno para encontrar filmes similares. Este método é eficaz quando temos muitos dados de avaliação, mas pode sofrer com o problema de "cold start" para novos filmes ou usuários.

2. **Recomendação Baseada em Conteúdo**: Utilizando os gêneros e características dos filmes. Este método funciona bem mesmo para novos filmes, desde que tenhamos informações sobre seu conteúdo.

Os dois métodos se complementam e podem ser usados juntos para criar sistemas de recomendação mais robustos e precisos.

Para uma aplicação mais completa, foi criado um script Python (`recomendador_interativo.py`) que permite ao usuário escolher filmes e receber recomendações personalizadas usando ambos os métodos.